### Description:

In this task you will work with a binary (located at ../bin) that implements AES-128 algorithm with a hardcoded key.

To run the binary you need to provide 16-bytes plaintext in hex, as following:
```
    ../aesenc --ptext="010203040506070809000a0b0c0d0e1f"
```    
The result of the binary is printed to console:
```
    2DD52BC59D4196FE961054A06ECE1260 0404050604030403050506050304040407
```    
where:
```
    2DD52BC59D4196FE961054A06ECE1260 is a ciphertext
    
    0404050604030403050506050304040407 is a trace
```
### Tips:

A 'trace' is a side-channel information left by a programmer to debug the implementation.

In this example a trace is a 17 bytes long array. The first byte of this array is a Hamming weight of the mask. The following 16 bytes are Hamming weights of the masked State bytes at the beginning of the 10th round.

### Task:

Your task is to find a Master key (round key 0) embedded into the binary. 

The master key is in the form of HEIG{XXXXXXXXXX}, where X is a ASCII printable symbol.

In [3]:
import numpy as np
import binascii
import random
import numpy.matlib
import matplotlib.pyplot as plt
import string
import re
import tqdm

import sca_training
import re
import socket

HOST = 'iict-mv330-sfa'
PORT = 4004
#----------------------------------------------------------------------------
# This function calls tested binary ../aesenc either with a user-defined 
# plaintext (if plaintext satisfies all the requirements) or with a 
# predefined plaintext (the same as in the header above)
#
# INPUTS:
#     plaintext - a string of 32 symbols representing 16 hex bytes of ciphertext
#     verbose   - a flag to print values in the function call or not
# OUTPUTS:
#     output  - a raw binary output
#     ctext   - resulted ciphertext converted to numpy array of uint8
#     trace   - a trace associated with encryption process
#----------------------------------------------------------------------------
def binary_aes128_encrypt(plaintext, verbose=False):
    import subprocess
    pattern = '[0-9A-F]{32,34}'
    
    #Check the correctness of the plaintext
    if all(c in string.hexdigits for c in plaintext) and len(plaintext) == 32:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((HOST, PORT))
            s.recv(1024)
            s.sendall(plaintext.encode())
            s.sendall(b'\n')

            output = s.recv(1024)

    
    #Get the ciphertext (first 32 symbols) and the trace (last 34 symbols)
    result = re.findall(pattern, str(output))
    #print(result[0], result[1])
    
    #Transform the result into numpy array
    ctext = np.frombuffer(binascii.unhexlify(result[0]), dtype=np.uint8)
    trace = np.frombuffer(binascii.unhexlify(result[1]), dtype=np.uint8)

    if (verbose):
        print('Binary output:', output.strip())
        print('Ciphertext as numpy array:', ctext)
        print('Trace as numpy array:', trace)
    
    return output, ctext, trace


#----------------------------------------------------------------------------
# This function calls supportive library to compute a master key from
# the last round key.
#
# INPUTS:
#     last_round_key - a numpy array of 16 elements representing 16 hex bytes of
#                      the last round key, i.e., Round Key 10
# OUTPUTS:
#     key_schedule  - 11 round keys of 16 bytes each (key_schedule[0,0,:] is the
#                     master key
#----------------------------------------------------------------------------
# key_schedule = sca_training.inverse_key_expansion(last_round_key)

In [4]:
output, ctext, trace = binary_aes128_encrypt('010203040506070809000a0b0c0d0e1f', verbose=True)

Binary output: b'2DD52BC59D4196FE961054A06ECE1260 0505040305040304020403020603030302'
Ciphertext as numpy array: [ 45 213  43 197 157  65 150 254 150  16  84 160 110 206  18  96]
Trace as numpy array: [5 5 4 3 5 4 3 4 2 4 3 2 6 3 3 3 2]


In [10]:
num_enc = 10000

traces = np.zeros((num_enc, 17), dtype=np.uint8)
ctexts = np.zeros((num_enc, 16), dtype=np.uint8)

for iEnc in tqdm.tqdm(range(num_enc)):
    ptext = '%032x' % random.randrange(16**32)
    output, ctext, trace = binary_aes128_encrypt(ptext, verbose=False)
    ctexts[iEnc,:] = ctext
    traces[iEnc,:] = trace
    
del output
del ctext
del trace

100%|████████████████████████████████████| 50000/50000 [00:55<00:00, 900.01it/s]


### ATTACK CODE
Here you need to implement your attack which gives you the last round key.

Once the last round key is found - you need to compute the master key: this can be done with the function get_master_key()

### Programming tips
#### Numpy code
* np.matlib.repmat(array, num_rows, num_cols)
* np.arange(256).astype(np.uint8)
* np.bitwise_xor(m1, m2)
* np.where()

#### Code prepared for training
* sca_training.invSbox[sbox_out]
* sca_training.HW_uint8[sbox_in]
* sca_training.shift_rows(traces)
* sca_training.compute_correlation()

#### Various cod
* ''.join('{:02x}'.format(c) for c in int_array)